##### Imports

In [2]:
import sqlite3
import pandas as pd

database_filename = "../gestionConferences.db"

## Requêtes

##### 1 - Liste des conférences qui ont lieu dans l'année courante

In [3]:
conn = sqlite3.connect(database_filename)
cursor = conn.cursor()

NOMBRE_DE_CONF_A_AFFICHER = 10
query = """ 
  SELECT id_conference, title, starting_date
  FROM CONFERENCE
  WHERE strftime('%Y', "starting_date") = strftime('%Y', 'now')
  ORDER BY starting_date ASC;
"""  

df = pd.read_sql(query, conn)
print(f"{NOMBRE_DE_CONF_A_AFFICHER} exemples de conférences ayant eu lieu dans l'année courante :")
for _, row in df.head(NOMBRE_DE_CONF_A_AFFICHER).iterrows():
  print(f"{row['starting_date']} : {row['title']} (id : {row['id_conference']})")
  
conn.close()

10 exemples de conférences ayant eu lieu dans l'année courante :
2025-01-16 : Challenges in Artificial Intelligence (id : 52)
2025-01-18 : Optimization of Climate Modeling (id : 33)
2025-01-19 : Modeling of Adaptive Robotics (id : 65)
2025-02-15 : Workshop : Exploring Adaptive Software Engineering (id : 93)
2025-02-17 : Optimization of Quantum Computing (id : 51)
2025-02-28 : Analysis of Robust Machine Learning (id : 36)
2025-03-08 : Optimization of Artificial Intelligence (id : 14)
2025-03-19 : Workshop : Evaluation of Adaptive Quantum Computing (id : 100)
2025-04-03 : Modeling of Robust Artificial Intelligence (id : 66)
2025-04-12 : Advances in Cybersecurity (id : 46)


##### 2 - Liste des pays, avec pour chaque pays la liste des conférences (titres) qui y ont été organisées

In [3]:
conn = sqlite3.connect(database_filename)
cursor = conn.cursor()

MAX_CONF_PAR_PAYS = 3
query = """
  SELECT country, title
  FROM CONFERENCE
  ORDER BY country;
"""

df = pd.read_sql(query, conn)
print(f"{MAX_CONF_PAR_PAYS} exemples de conférences pour chaque pays :\n")

for country, group in df.groupby("country"):
  nb_conf = len(group)
  print(f"{country} ({nb_conf} conférences en tout) :")
  for title in group["title"].head(MAX_CONF_PAR_PAYS):
    print(f"  - {title}")
  print()
conn.close()

3 exemples de conférences pour chaque pays :

Allemagne (7 conférences en tout) :
  - Analysis of Novel Robotics
  - Optimization of Robust Climate Modeling
  - Optimization of Cybersecurity

Espagne (13 conférences en tout) :
  - Analysis of Spatial Data Bases
  - Applications of Adaptive Robotics
  - Optimization of Data Mining

France (80 conférences en tout) :
  - Exploring Quantum Computing
  - Applications of Novel Cybersecurity
  - Challenges in Adaptive Robotics



##### 3 - Liste des conférences qui portent sur un domaine de recherche portant sur les bases de données spatiales (i.e. le mot clé entré dans la base de données n'est pas forcément "bases de données spatiales")

In [ ]:
conn = sqlite3.connect(database_filename)
cursor = conn.cursor()

NB_CONF_A_PRINT = 5

search_query = "bases data spatial" # requête de recherche
words = search_query.split()
where_condition_on_keywords = " OR ".join([f"key_words LIKE '%{word}%'" for word in words]) # OR key_words LIKE base OR key_words LIKE spatiale... 
query = f"""
  SELECT title, starting_date, key_words
  FROM CONFERENCE
  WHERE {where_condition_on_keywords}
  ORDER BY starting_date DESC;
"""

df = pd.read_sql(query, conn)
print(f"{NB_CONF_A_PRINT} exemples de conférences pour cette requête :\n")

for _, row in df.head(NB_CONF_A_PRINT).iterrows():
  print(f"{row['starting_date']} : {row['title']}, ----------- keywords = [{row['key_words']}]")
conn.close()



5 exemples de conférences pour cette requête :

2026-12-08 : Optimization of Spatial Data Bases, ----------- keywords = [data,spatial]
2026-11-16 : Evaluation of Adaptive Spatial Data Bases, ----------- keywords = [data,adaptive,spatial,evaluation]
2026-11-06 : Workshop : Optimization of Efficient Spatial Data Bases, ----------- keywords = [spatial,bases]
2026-07-05 : Optimization of Data Mining, ----------- keywords = [mining,data,optimization]
2026-02-01 : Exploring Spatial Data Bases, ----------- keywords = [spatial,bases]


##### 4 - Liste des conférences qui intéressent un utilisateur particulier

In [ ]:
conn = sqlite3.connect(database_filename)
cursor = conn.cursor()

NB_CONF_A_PRINT = 5
ID_USER = 2

cursor.execute( # profil utilisateur
  "SELECT profile FROM utilisateur WHERE id_user = ?;",
  (ID_USER,)
)
row = cursor.fetchone()

profile_keywords = [
  keyword.strip().lower()
  for keyword in row[0].split(",")
  if keyword.strip()
]

where_condition = " OR ".join(
  ["LOWER(key_words) LIKE ?"] * len(profile_keywords)
)
query = f"""
  SELECT title, starting_date, key_words
  FROM conference
  WHERE {where_condition}
  ORDER BY starting_date DESC
  LIMIT ?;
""" 

params = [f"%{kw}%" for kw in profile_keywords]
df = pd.read_sql(query, conn, params=params + [NB_CONF_A_PRINT])
conn.close()

print(f"Profil de l'utilisateur {ID_USER} :")
for i in range(len(profile_keywords)):
  print(f"- {profile_keywords[i]}")
print(f"\nConférences intéressantes pour l'utilisateur {ID_USER} :")
for _, row in df.iterrows():
  print(f"{row['starting_date']} : {row['title']} — [{row['key_words']}]")

Profil de l'utilisateur 2 :
- neural networks
- cybersecurity

Conférences intéressantes pour l'utilisateur 2 :
2026-07-04 : Optimization of Cybersecurity — [cybersecurity,optimization]
2026-04-24 : Workshop : Challenges in Adaptive Cybersecurity — [workshop,challenges,adaptive,cybersecurity]
2026-04-20 : Applications of Novel Cybersecurity — [cybersecurity,applications,novel]
2025-11-15 : Evaluation of Cybersecurity — [cybersecurity,evaluation]
2025-09-09 : Advances in Cybersecurity — [advances,cybersecurity]


##### 5 - Liste des conférences du groupe de conférence SDH

In [6]:
NB_CONF_A_PRINT = 5
conn = sqlite3.connect(database_filename)
SERIES_NAME = "SDH"

query = f"""
SELECT title, starting_date, city, country, series
FROM conference
WHERE series =?
ORDER BY starting_date DESC;
"""

df = pd.read_sql(query, conn, params=[SERIES_NAME])
conn.close()

print(f"{NB_CONF_A_PRINT} conférences du groupe {SERIES_NAME} :\n")

for i, row in df.iterrows():
  if i<NB_CONF_A_PRINT:
    print(f"{row['starting_date']} : {row['title']} ({row['city']}, {row['country']}) - {row['series']}")


5 conférences du groupe SDH :

2026-12-24 : Applications of Bioinformatics (Saint Matthieu, France) - SDH
2026-12-05 : Modeling of Quantum Computing (Goncalves, France) - SDH
2026-12-05 : Exploring Scalable Quantum Computing (Bègue, France) - SDH
2026-09-12 : Advances in Software Engineering (Schneider-sur-Mallet, France) - SDH
2026-09-02 : Applications of Cybersecurity (Leleuboeuf, France) - SDH


##### 6 - Liste des workshops organisés conjointement avec une conférence donnée

In [16]:
ID_CONF = 31  # conférence sélectionnée

conn = sqlite3.connect(database_filename)
query = """
SELECT id_conference, title, starting_date, ending_date
FROM conference
WHERE type = 'Workshop'
  AND associated_conf = ?;
"""

df = pd.read_sql(query, conn, params=(ID_CONF,))
conn.close()

print(f"Workshops organisés conjointement avec la conférence {ID_CONF} :\n")

if df.empty:
  print("Aucun workshop associé.")
else:
  for _, row in df.iterrows():
    print(f"- {row['title']} ({row['starting_date']} au {row['ending_date']})")


Workshops organisés conjointement avec la conférence 31 :

- Workshop : Advances in Software Engineering (2025-09-18 au 2025-09-22)


##### 7 - Liste des conférences qui ont publié dans les actes d'un éditeur donné

In [21]:
NB_CONF_A_PRINT = 5
EDITOR_NAME = "ACM"  # éditeur recherché (parmi "Springer", "Elsevier", "IEEE", "ACM", "Wiley", "Taylor & Francis", et "Nature")

conn = sqlite3.connect(database_filename)

query = """
SELECT title, starting_date, city, country
FROM conference
WHERE editor = ?
ORDER BY starting_date DESC;
"""

df = pd.read_sql(query, conn, params=(EDITOR_NAME,))
conn.close()

print(f"Conférences publiées chez l'éditeur {EDITOR_NAME} (total de {df.shape[0]} conférences trouvées):\n")

if df.empty:
  print("Aucune conférence trouvée.")
else:
  for _, row in df.iterrows():
    print(f"{row['starting_date']} : {row['title']} ({row['city']}, {row['country']})")


Conférences publiées chez l'éditeur ACM (total de 15 conférences trouvées):

2026-12-27 : Modeling of Efficient Climate Modeling (RocheBourg, France)
2026-11-24 : Challenges in High-Performance Quantum Computing (Dos Santos-sur-Vallet, France)
2026-09-02 : Applications of Cybersecurity (Leleuboeuf, France)
2026-06-22 : Workshop : Exploring Climate Modeling (Meunier-la-Forêt, France)
2026-04-10 : Optimization of Neural Networks (Bertrand, France)
2026-03-18 : Challenges in Climate Modeling (Salamanca, Espagne)
2026-01-28 : Optimization of Scalable Spatial Data Bases (Benardboeuf, France)
2025-10-12 : Workshop : Challenges in Robust Quantum Computing (Bourgeois, France)
2025-10-11 : Evaluation of Climate Modeling (Laroche, France)
2025-07-04 : Advances in Scalable Quantum Computing (Sainte Agatheboeuf, France)
2025-06-05 : Workshop : Exploring Bioinformatics (CordierBourg, France)
2025-05-25 : Optimization of Efficient Artificial Intelligence (Gallet, France)
2025-04-23 : Exploring Effic

##### 8 - Liste des utilisateurs de la base de données qui ont aussi été responsables d'une conférence (comme responsable ou membre du comité de programme)

In [26]:
conn = sqlite3.connect(database_filename)

query = """
SELECT DISTINCT user.id_user, user.profile, user.id_personne, respo.id_responsable, pers.name, pers.first_name
FROM utilisateur user
JOIN responsable respo ON user.id_personne = respo.id_personne
JOIN personne pers ON user.id_personne = pers.id_personne
ORDER BY user.id_user;
"""

df = pd.read_sql(query, conn)
conn.close()

print("Utilisateurs qui ont également été responsables :\n")

if df.empty:
  print("Aucun utilisateur responsable trouvé.")
else:
  for _, row in df.iterrows():
    print(f"{row['first_name']} {row['name']} - (id personne = {row['id_personne']}, id responsable = {row['id_responsable']}, id_user = {row['id_user']}) ")


Utilisateurs qui ont également été responsables :

William Sims - (id personne = 2, id responsable = 4, id_user = 6) 
Jaime Williams - (id personne = 46, id responsable = 10, id_user = 11) 
Shelby Williams - (id personne = 38, id responsable = 13, id_user = 14) 
Mark Bailey - (id personne = 21, id responsable = 8, id_user = 15) 
Austin Hancock - (id personne = 13, id responsable = 1, id_user = 16) 
Kimberly Wood - (id personne = 18, id responsable = 7, id_user = 21) 
Melinda Avery - (id personne = 42, id responsable = 6, id_user = 26) 
Linda Stokes - (id personne = 12, id responsable = 2, id_user = 27) 
Stephanie Eaton - (id personne = 32, id responsable = 11, id_user = 28) 
Roberto Fisher - (id personne = 5, id responsable = 15, id_user = 29) 
James Ochoa - (id personne = 14, id responsable = 9, id_user = 30) 
Jennifer Smith - (id personne = 50, id responsable = 12, id_user = 39) 


##### 9 - Liste des conférences pour lesquelles le professeur "X" a été responsable (comme responsable ou membre du comité de programme)

In [27]:
prof_name = "Sims"    # remplacer par nom du professeur
prof_first_name = "William"   # remplacer par prénom

conn = sqlite3.connect(database_filename)

query = """
SELECT DISTINCT c.id_conference, c.title, c.starting_date, c.city, c.country
FROM conference c
JOIN direction d ON c.id_conference = d.id_conference
JOIN responsable r ON d.id_responsable = r.id_responsable
JOIN personne p ON r.id_personne = p.id_personne
WHERE p.name = ? AND p.first_name = ?
ORDER BY c.starting_date DESC;
"""

df = pd.read_sql(query, conn, params=(prof_name, prof_first_name))
conn.close()

print(f"Conférences pour lesquelles le professeur {prof_first_name} {prof_name} a été responsable :\n")

if df.empty:
  print("Aucune conférence trouvée.")
else:
  for _, row in df.iterrows():
    print(f"{row['starting_date']} : {row['title']} ({row['city']}, {row['country']})")


Conférences pour lesquelles le professeur William Sims a été responsable :

2026-12-16 : Workshop : Advances in Adaptive Machine Learning (Saint MargauxVille, France)
2026-11-24 : Workshop : Modeling of Artificial Intelligence (Andre, France)
2026-09-02 : Applications of Cybersecurity (Leleuboeuf, France)
2026-06-15 : Advances in Novel Data Mining (Petit-sur-Fabre, France)
2026-05-11 : Workshop : Analysis of Novel Bioinformatics (Vizcaya, Espagne)
2026-04-19 : Workshop : Modeling of High-Performance Robotics (Ferrand, France)
2026-04-10 : Optimization of Neural Networks (Bertrand, France)
2026-03-20 : Workshop : Analysis of Neural Networks (Guipúzcoa, Espagne)
2025-11-05 : Applications of High-Performance Artificial Intelligence (Saint Hélène, France)
2025-10-24 : Analysis of Spatial Data Bases (La Coruña, Espagne)
2025-09-18 : Workshop : Advances in Software Engineering (Maillet-sur-Maréchal, France)
2025-07-05 : Exploring Bioinformatics (Wiedenbrück, Allemagne)
2025-07-05 : Analysis 